## **演示1203：DecisionTreeClassifier**

### **案例1：使用sklearn.tree.DecisionTreeClassifier对应聘人员数据分类**
* 在进行训练之前，要把字符串形式的Feature值转换成数值形式；一个特征下的不同特征取值，分别用不同的数值代表
* 字符串形式的Label也要转换成数值形式的分类
* 本例提供了14条训练数据和2条验证数据。请注意，验证数据也需要进行数值化转换。因此，本例中先将所有数据都进行数值化转换；但最后执行训练时，仅取前14条
* 本例的数值化转换代码暂未考虑验证数据中特征值缺失的问题！

In [1]:
''' 对数据进行数值化处理，再使用DecisionTreeClassifier进行决策 '''
import numpy as np    
import collections as col
from sklearn import tree

inputs = [
({'level':'Senior', 'lang':'Java', 'tweets':'no', 'phd':'no'}, False),
({'level':'Senior', 'lang':'Java', 'tweets':'no', 'phd':'yes'}, False),
({'level':'Mid', 'lang':'Python', 'tweets':'no', 'phd':'no'}, True),
({'level':'Junior', 'lang':'Python', 'tweets':'no', 'phd':'no'}, True),
({'level':'Junior', 'lang':'R', 'tweets':'yes', 'phd':'no'}, True),
({'level':'Junior', 'lang':'R', 'tweets':'yes', 'phd':'yes'}, False),
({'level':'Mid', 'lang':'R', 'tweets':'yes', 'phd':'yes'}, True),
({'level':'Senior', 'lang':'Python', 'tweets':'no', 'phd':'no'}, False),
({'level':'Senior', 'lang':'R', 'tweets':'yes', 'phd':'no'}, True),
({'level':'Junior', 'lang':'Python', 'tweets':'yes', 'phd':'no'}, True),
({'level':'Senior', 'lang':'Python', 'tweets':'yes', 'phd':'yes'}, True),
({'level':'Mid', 'lang':'Python', 'tweets':'no', 'phd':'yes'}, True),
({'level':'Mid', 'lang':'Java', 'tweets':'yes', 'phd':'no'}, True),
({'level':'Junior', 'lang':'Python', 'tweets':'no', 'phd':'yes'}, False),
#下列2行数据是用来进行预测的，请不要作为训练数据
({ "level" : "Junior", "lang" : "Java", "tweets" : "yes", "phd" : "no"}, True),
({ "level" : "Junior", "lang" : "Java", "tweets" : "yes", "phd" : "yes"}, False),
]

data = [row[0] for row in inputs]               # 仅获取特征矩阵(连同最后两行)
y = [1 if row[1] else 0 for row in inputs[:-2]] # 以0或1来代替True或False, 去掉最后2行用于预测的数据

# 以下：x中的每一列，列中的值统一转换成数值类型
rows_count = len(data)
columns_count = len(data[0])                    # 获取特征列的数目(不计Category列)
x = np.ndarray((rows_count, columns_count))     # 用于存放数值化之后的特征矩阵
column_index = 0
for column_key in data[0].keys():    # 对于每一个特征名
    column_counter = col.Counter([row[column_key] for row in data])         # 获取名为key的列的取值计数信息
    resetting_value = 1    # 用于针对每个key分别从1开始计数
    for counter_key in column_counter.keys():
        column_counter[counter_key] = resetting_value                       # 每种取值分别对应到1,2,3...
        resetting_value += 1
    for i in np.arange(rows_count):
        x[i,column_index] = column_counter[data[i][column_key]]             # 将该列中的每个数据分别映射到设置为1,2,3... 
    column_index += 1

print("数值映射后的特征矩阵：", x)
  
clf = tree.DecisionTreeClassifier(criterion = 'entropy') # CART算法，使用entropy作为标准；默认是是用gini作为标准 
clf = clf.fit(x[:-2], y)            # 最后2行预测数据要去掉
# 针对训练样本数据的准确率为100%
accuracy = clf.score(x[:-2], y)
print("针对训练数据的判别准确率：", accuracy)
predicts = clf.predict(x[-2:])
print("预测", inputs[-2], "结果：", predicts[0])
print("预测", inputs[-1], "结果：", predicts[1])

数值映射后的特征矩阵： [[1. 1. 1. 1.]
 [1. 1. 1. 2.]
 [2. 2. 1. 1.]
 [3. 2. 1. 1.]
 [3. 3. 2. 1.]
 [3. 3. 2. 2.]
 [2. 3. 2. 2.]
 [1. 2. 1. 1.]
 [1. 3. 2. 1.]
 [3. 2. 2. 1.]
 [1. 2. 2. 2.]
 [2. 2. 1. 2.]
 [2. 1. 2. 1.]
 [3. 2. 1. 2.]
 [3. 1. 2. 1.]
 [3. 1. 2. 2.]]
针对训练数据的判别准确率： 1.0
预测 ({'level': 'Junior', 'lang': 'Java', 'tweets': 'yes', 'phd': 'no'}, True) 结果： 1
预测 ({'level': 'Junior', 'lang': 'Java', 'tweets': 'yes', 'phd': 'yes'}, False) 结果： 0


### **案例2：使用sklearn.feature_extraction.DictVectorizer辅助进行数值化转化**
* 该对象能够将字符串形式的特征值提取出来，展开成数值向量形式
* 请注意，对于缺失特征值的情形，可能给出不同于以上案例的判别结果

In [3]:
''' 使用DictVectorizer进行数据预处理，再使用DecisionTreeClassifier决策树来判定 '''

import numpy as np    
import collections as col
from sklearn import tree
from sklearn.feature_extraction import DictVectorizer 

inputs = [
({'level':'Senior', 'lang':'Java', 'tweets':'no', 'phd':'no'}, False),
({'level':'Senior', 'lang':'Java', 'tweets':'no', 'phd':'yes'}, False),
({'level':'Mid', 'lang':'Python', 'tweets':'no', 'phd':'no'}, True),
({'level':'Junior', 'lang':'Python', 'tweets':'no', 'phd':'no'}, True),
({'level':'Junior', 'lang':'R', 'tweets':'yes', 'phd':'no'}, True),
({'level':'Junior', 'lang':'R', 'tweets':'yes', 'phd':'yes'}, False),
({'level':'Mid', 'lang':'R', 'tweets':'yes', 'phd':'yes'}, True),
({'level':'Senior', 'lang':'Python', 'tweets':'no', 'phd':'no'}, False),
({'level':'Senior', 'lang':'R', 'tweets':'yes', 'phd':'no'}, True),
({'level':'Junior', 'lang':'Python', 'tweets':'yes', 'phd':'no'}, True),
({'level':'Senior', 'lang':'Python', 'tweets':'yes', 'phd':'yes'}, True),
({'level':'Mid', 'lang':'Python', 'tweets':'no', 'phd':'yes'}, True),
({'level':'Mid', 'lang':'Java', 'tweets':'yes', 'phd':'no'}, True),
({'level':'Junior', 'lang':'Python', 'tweets':'no', 'phd':'yes'}, False),
# 后4行数据用于预测结果
({ "level" : "Junior", "lang" : "Java", "tweets" : "yes", "phd" : "no"}, True),
({ "level" : "Junior", "lang" : "Java", "tweets" : "yes", "phd" : "yes"}, False),
({ "level" : "Intern" }, True),
({ "level" : "Senior" }, False)
]

# 对Dict形式的字符特征值进行数值化转换(转成数值向量)
inputs_data = [row[0] for row in inputs]
y = [1 if row[1] else 0 for row in inputs]
vec = DictVectorizer()  
x = vec.fit_transform(inputs_data).toarray()
print("DictVectorizer转换后的特征矩阵中的特征名称：\n", vec.feature_names_)
print("特征矩阵值：\n", x)

seperate_index = -4
train_x = x[:seperate_index]
train_y = y[:seperate_index]

clf = tree.DecisionTreeClassifier(criterion = 'entropy')
clf.fit(train_x, train_y)
accuracy = clf.score(train_x, train_y)
print("针对训练数据的判别准确率：", accuracy)

# 对待预测数据进行数值化转换
test_x = x[seperate_index:]
predicts = clf.predict(test_x)
print("预测结果:")
print(predicts)

DictVectorizer转换后的特征矩阵中的特征名称：
 ['lang=Java', 'lang=Python', 'lang=R', 'level=Intern', 'level=Junior', 'level=Mid', 'level=Senior', 'phd=no', 'phd=yes', 'tweets=no', 'tweets=yes']
特征矩阵值：
 [[1. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0.]
 [0. 1. 0. 0. 0. 1. 0. 1. 0. 1. 0.]
 [0. 1. 0. 0. 1. 0. 0. 1. 0. 1. 0.]
 [0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 1.]
 [0. 0. 1. 0. 1. 0. 0. 0. 1. 0. 1.]
 [0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 1.]
 [0. 1. 0. 0. 0. 0. 1. 1. 0. 1. 0.]
 [0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 1.]
 [0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0. 1. 0. 1. 0. 1.]
 [0. 1. 0. 0. 0. 1. 0. 0. 1. 1. 0.]
 [1. 0. 0. 0. 0. 1. 0. 1. 0. 0. 1.]
 [0. 1. 0. 0. 1. 0. 0. 0. 1. 1. 0.]
 [1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 1.]
 [1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]]
针对训练数据的判别准确率： 1.0
预测结果:
[1 0 1 1]


### **案例3：使用DecisionTreeClassifier处理汽车数据分类**

In [3]:
''' 配合DictVectorizer和DecisionTreeClassifier对汽车数据进行判别 '''

import numpy as np    
from sklearn import tree    
from sklearn.feature_extraction import DictVectorizer 

data   = []    
labels = []    
with open("car.csv") as ifile:
        first_line = True
        for line in ifile:
            if first_line:              # 跳过第一行(标题行)
                first_line = False
                continue
            #data需要是字典形式，因为之后需要使用DictVectorizer()修改字符串数据类型，以便符合DecisionTreeClassifier()  
            rowDict = {}
            tokens = line.strip().split(',')  
            rowDict['buying']=tokens[0]
            rowDict['maint']=tokens[1]  
            rowDict['doors']=tokens[2]  
            rowDict['persons']=tokens[3]  
            rowDict['lug_boot']=tokens[4]  
            rowDict['safety']=tokens[5]  
            data.append(rowDict)  
            labels.append(tokens[-1])   
x = np.array(data)  
labels = np.array(labels)    
y = np.zeros(labels.shape)  # 初始label全为0  
  
y[labels =='vgood']=1       # 当label等于这三种属性的话，设置为1。  
y[labels =='good']=1  
y[labels =='acc']=1  
  
vec = DictVectorizer()      # 转换字符串数据类型  
dx = vec.fit_transform(x).toarray()  

# 拆分成训练数据和测试数据
ratio = 0.75
xTrain = []
yTrain = []
xTest = []
yTest = []
features = xTrain,xTest
labels = yTrain, yTest
for i in range(len(dx)):
    dataSetIndex = 0 if np.random.random() < ratio else 1
    features[dataSetIndex].append(dx[i])
    labels[dataSetIndex].append(y[i])
  
clf = tree.DecisionTreeClassifier(criterion = 'entropy') # CART算法，使用entropy作为标准；默认是是用gini作为标准 
clf.fit(xTrain,yTrain)        

# 检查准确率
accuracy = clf.score(xTest, yTest)
print(accuracy)

0.9832935560859188
